---
title: "Interactive Widgets in Jupyter"
subtitle: "Building Interactive Interfaces with ipywidgets"
---

## Introduction to Jupyter Widgets

Jupyter widgets (ipywidgets) allow you to build interactive GUIs for your notebooks. They're perfect for:

- **Data Exploration**: Interactive parameter tuning
- **Teaching**: Demonstrating concepts dynamically
- **Dashboards**: Simple data apps without leaving Jupyter
- **Model Tuning**: Real-time hyperparameter adjustment

This tutorial covers everything from basic widgets to complex interactive applications.

## Setup and Installation

In [ ]:
# Install if needed (uncomment)
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension

# Import libraries
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display, clear_output, HTML

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib_inline.backend_inline import set_matplotlib_formats

# Setup
set_matplotlib_formats('retina')
sns.set_theme(style='whitegrid')
plt.rcParams['figure.figsize'] = (8, 5)

print("✅ Widgets ready!")

## Part 1: Basic Widgets

### Simple Input Widgets

In [ ]:
# Text input
text = widgets.Text(
    value='Hello World',
    placeholder='Type something',
    description='String:',
    disabled=False
)
display(text)

In [ ]:
# Numeric input
int_slider = widgets.IntSlider(
    value=7,
    min=0,
    max=10,
    step=1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(int_slider)

In [ ]:
# Float slider with style
float_slider = widgets.FloatSlider(
    value=5.5,
    min=0,
    max=10.0,
    step=0.1,
    description='Float:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)
display(float_slider)

### Selection Widgets

In [ ]:
# Dropdown
dropdown = widgets.Dropdown(
    options=['Option 1', 'Option 2', 'Option 3'],
    value='Option 2',
    description='Choose:',
    disabled=False,
)

# Radio buttons
radio = widgets.RadioButtons(
    options=['Small', 'Medium', 'Large'],
    value='Medium',
    description='Size:',
    disabled=False
)

# Checkbox
checkbox = widgets.Checkbox(
    value=False,
    description='Check me',
    disabled=False,
    indent=False
)

# Display all
display(dropdown, radio, checkbox)

### Button and Output Widgets

In [ ]:
# Button with callback
button = widgets.Button(
    description='Click me!',
    disabled=False,
    button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to see magic',
    icon='check'  # FontAwesome icon
)

output = widgets.Output()

def on_button_click(b):
    with output:
        clear_output()
        print(f"Button clicked! Value from slider: {int_slider.value}")

button.on_click(on_button_click)
display(button, output)

## Part 2: The `interact` Function

The easiest way to create interactive widgets!

In [ ]:
# Simple interact
@interact(x=(0, 10), y=(0, 10))
def add(x, y):
    return f"{x} + {y} = {x + y}"

In [ ]:
# Interact with plot
@interact(freq=(1, 10, 0.5), amplitude=(0.1, 2, 0.1), phase=(0, 2*np.pi, 0.1))
def plot_sine(freq=2, amplitude=1, phase=0):
    x = np.linspace(0, 2*np.pi, 1000)
    y = amplitude * np.sin(freq * x + phase)
    
    plt.figure(figsize=(10, 4))
    plt.plot(x, y, 'b-', linewidth=2)
    plt.ylim(-2.5, 2.5)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title(f'y = {amplitude:.1f} * sin({freq:.1f}x + {phase:.1f})')
    plt.grid(True, alpha=0.3)
    plt.show()

### Automatic Widget Generation

In [ ]:
# Interact automatically creates appropriate widgets
@interact
def auto_widgets(
    text="Hello",                    # Creates Text widget
    number=5,                        # Creates IntSlider
    float_num=5.5,                   # Creates FloatSlider
    boolean=True,                    # Creates Checkbox
    options=['A', 'B', 'C'],        # Creates Dropdown
    date=(1, 100, 10)               # Creates IntSlider with range
):
    print(f"Text: {text}")
    print(f"Number: {number}")
    print(f"Float: {float_num}")
    print(f"Boolean: {boolean}")
    print(f"Option: {options}")
    print(f"Date: {date}")

## Part 3: Interactive Data Exploration

In [ ]:
# Load sample data
penguins = sns.load_dataset('penguins')
penguins_clean = penguins.dropna()

# Interactive data explorer
@interact(
    x_col=list(penguins_clean.select_dtypes(include=[np.number]).columns),
    y_col=list(penguins_clean.select_dtypes(include=[np.number]).columns),
    hue_col=['None'] + list(penguins_clean.columns),
    plot_type=['scatter', 'line', 'hex']
)
def explore_data(x_col='bill_length_mm', y_col='bill_depth_mm', 
                 hue_col='species', plot_type='scatter'):
    
    plt.figure(figsize=(10, 6))
    
    if hue_col == 'None':
        hue_data = None
    else:
        hue_data = penguins_clean[hue_col]
    
    if plot_type == 'scatter':
        sns.scatterplot(data=penguins_clean, x=x_col, y=y_col, hue=hue_data, s=100)
    elif plot_type == 'line':
        if hue_data is not None:
            for val in penguins_clean[hue_col].unique():
                data = penguins_clean[penguins_clean[hue_col] == val]
                plt.plot(data[x_col], data[y_col], 'o-', label=val, alpha=0.7)
            plt.legend()
        else:
            plt.plot(penguins_clean[x_col], penguins_clean[y_col], 'o-')
    elif plot_type == 'hex':
        plt.hexbin(penguins_clean[x_col], penguins_clean[y_col], gridsize=20, cmap='YlOrRd')
        plt.colorbar(label='Count')
    
    plt.xlabel(x_col)
    plt.ylabel(y_col)
    plt.title(f'{plot_type.capitalize()} Plot: {x_col} vs {y_col}')
    plt.tight_layout()
    plt.show()
    
    # Show statistics
    print(f"\nCorrelation: {penguins_clean[x_col].corr(penguins_clean[y_col]):.3f}")
    print(f"Data points: {len(penguins_clean)}")

## Part 4: Layout and Styling

### Container Widgets

In [ ]:
# HBox - Horizontal layout
slider1 = widgets.IntSlider(description='A')
slider2 = widgets.IntSlider(description='B')
slider3 = widgets.IntSlider(description='C')

hbox = widgets.HBox([slider1, slider2, slider3])
display(hbox)

In [ ]:
# VBox - Vertical layout
vbox = widgets.VBox([
    widgets.Label('Vertical Stack:'),
    widgets.IntSlider(description='Value 1'),
    widgets.IntSlider(description='Value 2'),
    widgets.IntSlider(description='Value 3')
])
display(vbox)

In [ ]:
# Tabs
tab1 = widgets.VBox([widgets.HTML('<h3>Settings</h3>'),
                     widgets.IntSlider(description='Speed'),
                     widgets.FloatSlider(description='Quality')])

tab2 = widgets.VBox([widgets.HTML('<h3>Advanced</h3>'),
                     widgets.Checkbox(description='Enable feature'),
                     widgets.Dropdown(options=['Mode 1', 'Mode 2'])])

tabs = widgets.Tab(children=[tab1, tab2])
tabs.set_title(0, 'Basic')
tabs.set_title(1, 'Advanced')
display(tabs)

In [ ]:
# Accordion
accordion = widgets.Accordion(children=[
    widgets.IntSlider(description='Setting 1'),
    widgets.Text(description='Setting 2'),
    widgets.Dropdown(options=['A', 'B', 'C'], description='Setting 3')
])
accordion.set_title(0, 'Numeric Settings')
accordion.set_title(1, 'Text Settings')
accordion.set_title(2, 'Choice Settings')
display(accordion)

### Custom Styling

In [ ]:
# Styled widgets
styled_slider = widgets.IntSlider(
    value=50,
    min=0,
    max=100,
    description='Styled:',
    style={
        'description_width': '100px',
        'handle_color': 'lightblue'
    },
    layout=widgets.Layout(
        width='80%',
        height='50px',
        border='2px solid blue',
        padding='10px',
        margin='10px'
    )
)

styled_button = widgets.Button(
    description='Styled Button',
    button_style='info',
    layout=widgets.Layout(
        width='200px',
        height='50px'
    )
)

display(styled_slider, styled_button)

## Part 5: Advanced Interactive Applications

### Interactive Machine Learning Model

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# Generate sample data
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, 
                          n_redundant=5, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Interactive model training
@interact(
    n_estimators=(10, 200, 10),
    max_depth=(1, 20),
    min_samples_split=(2, 20),
    criterion=['gini', 'entropy']
)
def train_model(n_estimators=100, max_depth=10, min_samples_split=2, criterion='gini'):
    # Train model
    clf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        criterion=criterion,
        random_state=42
    )
    clf.fit(X_train, y_train)
    
    # Predictions
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    # Visualization
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax1)
    ax1.set_title(f'Confusion Matrix\nAccuracy: {accuracy:.3f}')
    ax1.set_xlabel('Predicted')
    ax1.set_ylabel('Actual')
    
    # Feature importance
    importance = clf.feature_importances_
    indices = np.argsort(importance)[::-1][:10]
    ax2.bar(range(10), importance[indices])
    ax2.set_title('Top 10 Feature Importances')
    ax2.set_xlabel('Feature Index')
    ax2.set_ylabel('Importance')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📊 Model Performance:")
    print(f"Training samples: {len(X_train)}")
    print(f"Test samples: {len(X_test)}")
    print(f"Accuracy: {accuracy:.3f}")

### Interactive Image Processing

In [ ]:
from scipy import ndimage
from skimage import data

# Load sample image
image = data.camera()

@interact(
    blur=(0, 10, 0.5),
    rotation=(-180, 180, 10),
    brightness=(-100, 100, 10),
    contrast=(0.5, 2, 0.1)
)
def process_image(blur=0, rotation=0, brightness=0, contrast=1):
    # Process image
    processed = image.copy().astype(float)
    
    # Apply blur
    if blur > 0:
        processed = ndimage.gaussian_filter(processed, blur)
    
    # Apply rotation
    if rotation != 0:
        processed = ndimage.rotate(processed, rotation, reshape=False)
    
    # Adjust brightness and contrast
    processed = (processed - 128) * contrast + 128 + brightness
    processed = np.clip(processed, 0, 255)
    
    # Display
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    ax1.imshow(image, cmap='gray')
    ax1.set_title('Original')
    ax1.axis('off')
    
    ax2.imshow(processed, cmap='gray', vmin=0, vmax=255)
    ax2.set_title('Processed')
    ax2.axis('off')
    
    plt.tight_layout()
    plt.show()

## Part 6: Building a Complete Dashboard

In [ ]:
# Complete interactive dashboard
class DataDashboard:
    def __init__(self, data):
        self.data = data
        self.setup_widgets()
        self.setup_layout()
        
    def setup_widgets(self):
        # Data selection
        self.column_x = widgets.Dropdown(
            options=self.data.select_dtypes(include=[np.number]).columns,
            description='X axis:'
        )
        self.column_y = widgets.Dropdown(
            options=self.data.select_dtypes(include=[np.number]).columns,
            description='Y axis:'
        )
        self.color_by = widgets.Dropdown(
            options=['None'] + list(self.data.columns),
            description='Color by:'
        )
        
        # Plot type
        self.plot_type = widgets.RadioButtons(
            options=['Scatter', 'Box', 'Histogram', 'Correlation'],
            description='Plot Type:'
        )
        
        # Update button
        self.update_button = widgets.Button(
            description='Update Plot',
            button_style='success'
        )
        self.update_button.on_click(self.update_plot)
        
        # Output area
        self.output = widgets.Output()
        
        # Statistics text
        self.stats_output = widgets.Output()
        
    def setup_layout(self):
        # Control panel
        controls = widgets.VBox([
            widgets.HTML('<h3>Data Selection</h3>'),
            self.column_x,
            self.column_y,
            self.color_by,
            widgets.HTML('<h3>Visualization</h3>'),
            self.plot_type,
            self.update_button
        ])
        
        # Main layout
        self.dashboard = widgets.HBox([
            controls,
            widgets.VBox([
                self.output,
                self.stats_output
            ])
        ])
        
    def update_plot(self, button):
        with self.output:
            clear_output(wait=True)
            
            fig, ax = plt.subplots(figsize=(10, 6))
            
            if self.plot_type.value == 'Scatter':
                hue = None if self.color_by.value == 'None' else self.data[self.color_by.value]
                sns.scatterplot(data=self.data, 
                              x=self.column_x.value, 
                              y=self.column_y.value,
                              hue=hue, ax=ax)
                
            elif self.plot_type.value == 'Box':
                self.data.boxplot(column=self.column_y.value, ax=ax)
                
            elif self.plot_type.value == 'Histogram':
                ax.hist(self.data[self.column_x.value].dropna(), bins=30, edgecolor='black')
                ax.set_xlabel(self.column_x.value)
                
            elif self.plot_type.value == 'Correlation':
                corr_matrix = self.data.select_dtypes(include=[np.number]).corr()
                sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, ax=ax)
            
            plt.tight_layout()
            plt.show()
        
        # Update statistics
        with self.stats_output:
            clear_output(wait=True)
            if self.plot_type.value == 'Scatter':
                corr = self.data[self.column_x.value].corr(self.data[self.column_y.value])
                print(f"Correlation: {corr:.3f}")
                print(f"Data points: {len(self.data.dropna())}")
    
    def display(self):
        display(self.dashboard)
        self.update_plot(None)  # Initial plot

# Create and display dashboard
dashboard = DataDashboard(penguins)
dashboard.display()

## Part 7: Linking Widgets

In [ ]:
# Linked widgets example
a = widgets.FloatSlider(description='a', min=-10, max=10, value=1)
b = widgets.FloatSlider(description='b', min=-10, max=10, value=0)
c = widgets.FloatSlider(description='c', min=-10, max=10, value=0)

def update_equation(change):
    with equation_output:
        clear_output(wait=True)
        
        x = np.linspace(-10, 10, 1000)
        y = a.value * x**2 + b.value * x + c.value
        
        plt.figure(figsize=(10, 6))
        plt.plot(x, y, 'b-', linewidth=2)
        plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
        plt.axvline(x=0, color='k', linestyle='-', alpha=0.3)
        plt.grid(True, alpha=0.3)
        plt.xlim(-10, 10)
        plt.ylim(-50, 50)
        plt.title(f'y = {a.value:.1f}x² + {b.value:.1f}x + {c.value:.1f}')
        plt.xlabel('x')
        plt.ylabel('y')
        plt.show()

# Link widgets to function
a.observe(update_equation, names='value')
b.observe(update_equation, names='value')
c.observe(update_equation, names='value')

equation_output = widgets.Output()

# Display
display(widgets.VBox([a, b, c, equation_output]))
update_equation(None)  # Initial plot

## Part 8: File Upload Widget

In [ ]:
# File upload widget
upload = widgets.FileUpload(
    accept='.csv',  # Accept CSV files
    multiple=False  # Single file
)

output = widgets.Output()

def handle_upload(change):
    with output:
        clear_output()
        if upload.value:
            # Get uploaded file
            uploaded_file = list(upload.value.values())[0]
            content = uploaded_file['content']
            
            # Read CSV
            import io
            df = pd.read_csv(io.BytesIO(content))
            
            print(f"File uploaded: {uploaded_file['metadata']['name']}")
            print(f"Shape: {df.shape}")
            print("\nFirst 5 rows:")
            display(df.head())
            
            # Simple plot
            if len(df.select_dtypes(include=[np.number]).columns) >= 2:
                fig, ax = plt.subplots(figsize=(10, 6))
                df.plot(kind='scatter', 
                       x=df.columns[0], 
                       y=df.columns[1], 
                       ax=ax)
                plt.show()

upload.observe(handle_upload, names='value')

display(widgets.VBox([
    widgets.HTML('<h3>Upload CSV File</h3>'),
    upload,
    output
]))

## Best Practices and Tips

### 1. Performance Optimization

- Use `continuous_update=False` for sliders to avoid excessive updates
- Cache expensive computations
- Use `interact_manual` for heavy computations
- Limit data size for real-time interactions

### 2. User Experience

- Provide clear labels and descriptions
- Use appropriate widget types for data
- Group related controls
- Add loading indicators for slow operations

### 3. Layout Guidelines

- Use VBox/HBox for simple layouts
- Use Grid for complex arrangements
- Keep controls organized and intuitive
- Provide visual feedback for actions

## Troubleshooting

### Common Issues

1. **Widgets not displaying**: 
   - Ensure `ipywidgets` is installed
   - Enable notebook extension: `jupyter nbextension enable --py widgetsnbextension`

2. **Updates not working**:
   - Check callback function signatures
   - Use `.observe()` for custom widgets
   - Verify widget value types

3. **Performance issues**:
   - Reduce update frequency
   - Optimize computation code
   - Use caching where possible

## Advanced Topics

### Custom Widget Classes

In [ ]:
# Create custom widget
class ColorPicker(widgets.HBox):
    def __init__(self):
        super().__init__()
        
        self.r = widgets.IntSlider(min=0, max=255, description='R')
        self.g = widgets.IntSlider(min=0, max=255, description='G')
        self.b = widgets.IntSlider(min=0, max=255, description='B')
        self.preview = widgets.HTML(
            value='<div style="width:100px;height:100px;background:rgb(0,0,0)"></div>'
        )
        
        self.r.observe(self.update_color, 'value')
        self.g.observe(self.update_color, 'value')
        self.b.observe(self.update_color, 'value')
        
        self.children = [self.r, self.g, self.b, self.preview]
    
    def update_color(self, change):
        color = f'rgb({self.r.value},{self.g.value},{self.b.value})'
        self.preview.value = f'<div style="width:100px;height:100px;background:{color}"></div>'
    
    @property
    def value(self):
        return (self.r.value, self.g.value, self.b.value)

# Use custom widget
color_picker = ColorPicker()
display(color_picker)

## Resources and Further Learning

### Documentation
- [ipywidgets Documentation](https://ipywidgets.readthedocs.io/)
- [Widget List](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html)
- [Layout and Styling](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Styling.html)

### Advanced Libraries
- **Voilà**: Turn notebooks into standalone web apps
- **Panel**: More advanced dashboarding
- **Dash**: Production-ready dashboards
- **Streamlit**: Rapid app development

### Examples
- [ipywidgets Gallery](https://github.com/jupyter-widgets/ipywidgets/tree/master/docs/source/examples)
- [Interactive Visualizations](https://github.com/jupyter-widgets/pythreejs)

## Summary

You've learned:
- **Basic Widgets**: Input, selection, and display widgets
- **Interact**: Quick interactive functions
- **Layouts**: Organizing widgets effectively
- **Linking**: Creating responsive interfaces
- **Dashboards**: Building complete applications

Jupyter widgets transform static notebooks into interactive applications, perfect for:
- Data exploration
- Teaching and demonstrations
- Prototype development
- Model tuning

Keep experimenting and building! 🎛️🎨📊